In [1]:
import numpy as np
from New_functions import *
from Benchmark_functions import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
'''
First, we generate a dataset, we can change the Alpha_s, Alpha_t and effect parameter to change the distribution of
the generated dataset.
'''

import numpy as np

def generate(ns, nt, p, q, s, t, u, Alpha_s=1, Alpha_t=0, effect=1, x_effect=0, z_diff=0.1, threshold_X=0.5, threshold_Y=0.5):
    # Generate normal distributions for Z
    Zs_null = np.random.normal(0, 1, (ns, q))
    Zt_null = np.random.normal(0, 1, (nt, q))
    
    # Generate Z variables with and without shift
    Z_source = np.hstack((np.random.normal(0, 1, (ns, p)), Zs_null))
    Z_target = np.hstack((np.random.normal(z_diff, 1, (nt, p)), Zt_null))
    
    # Generate X variables
    X_source = Z_source[:, :p] @ u + np.random.normal(0, 1, ns)
    X_target = Z_target[:, :p] @ u + np.random.normal(0, 1, nt)
    
    # Convert X to binary
    X_source = (np.random.rand(ns) < 1 / (1 + np.exp(-X_source))).astype(int)
    X_target = (np.random.rand(nt) < 1 / (1 + np.exp(-X_target))).astype(int)
    
    # Generate V variables
    V_source = Z_source[:, :p] @ s + Alpha_s * X_source + np.random.normal(0, 5, ns)
    V_target = Z_target[:, :p] @ t + Alpha_t * X_target + np.random.normal(0, 5, nt)
    
    # Generate Y variables
    Y_source = (Z_source[:, :p].sum(axis=1))**2 + effect * V_source + np.random.normal(0, 1, ns) + x_effect * X_source
    Y_target = (Z_target[:, :p].sum(axis=1))**2 + effect * V_target + np.random.normal(0, 1, nt) + x_effect * X_target
    
    # Convert Y to binary
    Y_source = (np.random.rand(ns) < 1 / (1 + np.exp(-Y_source))).astype(int)
    Y_target = (np.random.rand(nt) < 1 / (1 + np.exp(-Y_target))).astype(int)
    
    return Y_source.reshape(-1, 1), X_source.reshape(-1, 1), V_source.reshape(-1, 1), Z_source,\
           Y_target.reshape(-1, 1), X_target.reshape(-1, 1), V_target.reshape(-1, 1), Z_target

In [3]:
#Set parameter for the generation of data
ns, p,q = 1000, 5, 10
nt = 2000

s = np.array([-1, -0.5, 0, 1, 1.5])
t = np.array([ -1, -1, 0.5 , 0.5, 1])
u = np.array([ 0, -1, 0.5, -0.5, 1])

Y_source, X_source, V_source, Z_source,Y_target, X_target, V_target, Z_target = \
generate(ns,nt, p,q, s, t, u, Alpha_s=0, Alpha_t = 2,effect=2, z_diff = 0.1)
            

In [4]:
# Data Separation:
# The data arrays X_e, Z_e, V_e are designated for density ratio estimation.
# The arrays Z_source, X_source, V_source, Y_source are used for testing.
# Here, we split the source data based on a specified proportion.

proportion = 0.5
num = int(proportion * X_source.shape[0])
Z_e = Z_source[:num]
X_e = X_source[:num]
V_e = V_source[:num]
Z_source = Z_source[num+1:]
X_source = X_source[num+1:]
V_source = V_source[num+1:]
Y_source = Y_source[num+1:]


# Real data experiments to run

## Our method

In [7]:
#1. Test without power enhancement
Test(X_e, Z_e, V_e, X_source, Z_source, V_source, Y_source, \
     X_target, Z_target, V_target, L=3, K=20, datatype='binary')

Training accuracy for X|Z: 0.72
The weighted rank distribution: [164.31407168 171.13969916 206.86242439]


0.040717883434907676

In [8]:
#2. Test with power enhancement
Test_pe(X_e, Z_e, V_e, X_source, Z_source, V_source, Y_source, \
        X_target, Z_target, V_target, L=3, K=20, datatype='binary')

Training accuracy for X|Z: 0.72
[0.30856275 0.2065974  0.2344884 ]
[158.99085744 202.43587061 168.55025067]


0.07607153746787443

### Use a different scoring function for testing V for the Power enhancement

In [9]:
# Originally, we use v*x for scoring, here we use (-v)*x to detect 
# the negative correlation between Y and V
Test_pe(X_e, Z_e, V_e, X_source, Z_source, V_source, Y_source, \
        X_target, Z_target, V_target, L=3, K=20, datatype='binary', score = 'neg')

Training accuracy for X|Z: 0.72
[-0.17263056  0.20051239 -0.20537856]
[169.92216047 174.40369059 221.62214139]


0.006430736018661398

## Benchmark
there are 3 benchmarks:1. Use source only data, 2. Use target only data, 3. Importance sampling method (the benchmark from others)

In [5]:
#1. Use source data only
PCR_test(X_source,Z_source,V_source,Y_source)

Training accuracy for X|Z: 0.7108433734939759


0.04413589298362197

In [6]:
#2. Use target data only

PCR_test(X_target,Z_target,V_target,Y_target)

Training accuracy for X|Z: 0.719


0.14501313841163965

In [8]:
#3. Use Importance Sampling benchmark method
IS_test(X_e, Z_e, V_e, X_source, Z_source, V_source, Y_source, X_target, Z_target, V_target, L=3, K=20, datatype='binary')

Training accuracy for X|Z: 0.717


0.44020585002260737

## Tune hyperparameter L

In [9]:
l_lst = [2, 5, 8, 10]
result_lst = []
for l in l_lst:
    # Use any test function above
    pvalue = Test_pe(X_e, Z_e, V_e, X_source, Z_source, V_source, Y_source, X_target, Z_target, V_target, L=3, K=20, datatype='binary')
    result_lst.append(pvalue)
    print(f'L is {l}, pvalue: {pvalue}')

Training accuracy for X|Z: 0.717
[0.23433064 0.19167032 0.28155873]
[167.95443516 165.08707733 174.75729811]
L is 2, pvalue: 0.933563916771966
Training accuracy for X|Z: 0.717
[0.2952632  0.20613484 0.31138347]
[140.64749537 187.91719237 174.08845475]
L is 5, pvalue: 0.06342484042790109
Training accuracy for X|Z: 0.717
[0.2715509  0.20274784 0.29688373]
[148.2607481  172.03479922 184.74858996]
L is 8, pvalue: 0.21636110362760874
Training accuracy for X|Z: 0.717
[0.29121139 0.13614264 0.34778643]
[146.78635223 166.57257311 184.67772408]
L is 10, pvalue: 0.195118444312963


In [11]:
print(result_lst)

[0.933563916771966, 0.06342484042790109, 0.21636110362760874, 0.195118444312963]
